     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 63.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.84 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible

## 1. INSTALL LIBRARIES AND PACKAGES

In [1]:

# Import the necessary libraries
!pip install datasets
!pip install transformers
!pip install torch
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]
!pip install numpy
!pip install numpy==1.22.0


  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
  Obtaining dependency information for jiwer from https://files.pythonhosted.org/packages/0d/4f/ee537ab20144811dd99321735ff92ef2b3a3230b77ed7454bed4c44d21fc/jiwer-3.0.3-py3-none-any.whl.metadata


In [2]:
##Imports required
import numpy as np
from datasets import load_dataset, Audio, DatasetDict
import torch
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
#from transformers import TrainingArguments, Trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


cuda


## 2. IMPORT DATA

In [5]:
dataset = DatasetDict()

# Load the PolyAI dataset.

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:80]")

# Remove unnecessary columns


# Split the datasedataset = 
dataset.remove_columns(['path','english_transcription','intent_class','lang_id'])

#split into train and test
dataset = dataset.train_test_split(test_size = 0.2, shuffle=False)


In [ ]:
#dataset['train']['audio'][0]

In [8]:
## Resample the dataset to 16 Khz as MCTCT model is trained on 16khz
dataset['train'] = dataset['train'].cast_column("audio", Audio(sampling_rate=16000))
dataset['test'] = dataset['test'].cast_column("audio", Audio(sampling_rate=16000))
# #dataset['train'] = dataset['train'].rename_column("audio", Audio(sampling_rate=16000))
# dataset['train'] = dataset['train'].rename_column("transcription","sentence")
# dataset['test'] = dataset['test'].rename_column("transcription","sentence")

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 64
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 16
    })
})

## 3. DATA PREPROCESSING

In [10]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en",task="transcribe", model_max_length=225)
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-tiny.en')
model.to(device)


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

In [13]:
# Preparing a function to process the entire dataset
# We need to crate two variables with name 'input_featrues'(input array of sound wave in raw foram) and 'labels'(transcription)

def prepare_dataset(batch):
    audio = batch["audio"]

   # batch["input_ids"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]

    #processor.feature_extractor.pad(input_features, return_tensors="pt")
  #  with processor.as_target_processor():
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

In [15]:
np.object = object    
encoded_dataset = dataset.map(prepare_dataset,num_proc=4)

#0:   0%|          | 0/16 [00:00<?, ?ex/s]

#1:   0%|          | 0/16 [00:00<?, ?ex/s]

#2:   0%|          | 0/16 [00:00<?, ?ex/s]

#3:   0%|          | 0/16 [00:00<?, ?ex/s]

#0:   0%|          | 0/4 [00:00<?, ?ex/s]

#1:   0%|          | 0/4 [00:00<?, ?ex/s]

#3:   0%|          | 0/4 [00:00<?, ?ex/s]

#2:   0%|          | 0/4 [00:00<?, ?ex/s]

In [16]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_features', 'labels'],
        num_rows: 64
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_features', 'labels'],
        num_rows: 16
    })
})

In [17]:
# Creating a DataCollatorClass

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#     def __call__(self, features):# List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
       # print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
       # print(batch)
        return batch #batch


In [18]:

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [19]:

# Evalution metric- We will be evaluating our model on word error rate

import evaluate

metric = evaluate.load("wer")


def compute_metrics(pred):
 #   wer = evaluate.load("wer")
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


#### 4. Training the model

In [20]:

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
training_args = Seq2SeqTrainingArguments(
    output_dir="seqtoseq-trained",
    gradient_checkpointing=True,
    per_device_train_batch_size=2,
    learning_rate=1e-5,
    warmup_steps=2,
    max_steps=2000,
    fp16=True ,#False ,#True,
    optim='adafactor',
   # group_by_length=True,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    report_to = ["tensorboard"],
    #data_parallel=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #data_parallel=False
  #  sampler = None
)


In [22]:
# Requires GPU for training
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Wer
100,No log,0.661608,62.903226
200,No log,0.208757,374.193548
300,No log,0.233541,2024.193548
400,No log,0.337271,2896.774194
500,0.223400,0.382448,25.806452
600,0.223400,0.394426,479.032258
700,0.223400,0.416275,585.483871
800,0.223400,0.466375,674.193548
900,0.223400,0.468723,845.161290
1000,0.000000,0.469956,908.064516


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=2000, training_loss=0.05585742835774909, metrics={'train_runtime': 2698.7692, 'train_samples_per_second': 2.964, 'train_steps_per_second': 0.741, 'total_flos': 1.9695108096e+17, 'train_loss': 0.05585742835774909, 'epoch': 125.0})

In [ ]:
## getting test data
i2 = processor(dataset['test'][6]["audio"]["array"], sampling_rate=16000, return_tensors="pt")
print(f"The input test audio is: {dataset['test'][8]['transcription']}")

generated_ids = model.generate(inputs=input_features)
# prediction for test data
with torch.no_grad():
    logits = model(**i2.to(device)).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(f'The output prediction is : {transcription[0]}')
     

In [37]:
## getting test data
inputs = processor(dataset['test'][8]["audio"]["array"], sampling_rate=16000, return_tensors="pt").to(device).input_features
print(f"The input test audio is: {dataset['test'][8]['transcription']}")

generated_ids = model.generate(inputs=inputs)
# prediction for test data
# with torch.no_grad():
#     logits = model(**i2.to(device)).logits

#predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'The output prediction is : {transcription}')
     

The input test audio is: how much do I have in my account
The output prediction is :  I'm much do I have in my account
